In [ ]:
import numpy as np
import pandas as pd

In [ ]:
ratings = pd.read_csv("/tmp/amazon_ratings_Books.csv", sep=",", names=['user_id', 'item_id', 'rating', 'timestamp'])

In [ ]:
ratings.head()

In [ ]:
ratings.shape[0]

In [ ]:
len(set(ratings['user_id'].values))

In [ ]:
len(set(ratings['item_id'].values))

In [ ]:
item_id_count = ratings['item_id'].value_counts().rename_axis('unique_values').reset_index(name='item_count')
item_id_count.head()

In [ ]:
user_id_count = ratings['user_id'].value_counts().rename_axis('unique_values').reset_index(name='user_count')
user_id_count.head()

In [ ]:
ratings = ratings.join(item_id_count.set_index('unique_values'), on='item_id')
ratings.head()

In [ ]:
ratings = ratings.join(user_id_count.set_index('unique_values'), on='user_id')
ratings.head()

In [ ]:
ratings = ratings[(ratings['item_count'] >= 5) & (ratings['user_count'] >= 5)]
ratings.head()

In [ ]:
len(ratings)

In [ ]:
ratings.dtypes

In [ ]:
ratings['user_id'] = pd.Categorical(ratings['user_id'])
ratings['user_id'] = ratings['user_id'].cat.codes

ratings['item_id'] = pd.Categorical(ratings['item_id'])
ratings['item_id'] = ratings['item_id'].cat.codes
ratings.head()

In [ ]:
ratings.dtypes

In [ ]:
len(set(ratings['user_id'].values)), len(set(ratings['item_id'].values))

In [ ]:
ratings_group = ratings.sort_values(by=['timestamp']).groupby('user_id')
ratings_group.head()

In [ ]:
type(ratings_group)

In [ ]:
ratings_group.item_id.apply(list).head()

In [ ]:
seq_ratings_data = pd.DataFrame(
    data ={
        'user_id': list(ratings_group.groups.keys()),
        'item_ids': list(ratings_group.item_id.apply(list)),
        'ratings': list(ratings_group.rating.apply(list)),
        'timestamps': list(ratings_group.timestamp.apply(list))
    }
)

In [ ]:
seq_ratings_data.head()

In [ ]:
len(seq_ratings_data)

In [ ]:
seq_ratings_data = seq_ratings_data[seq_ratings_data['item_ids'].apply(len) >= 5]
len(seq_ratings_data)

In [ ]:
result = pd.DataFrame([[]])
for col in ['item_ids']:
    seq_ratings_len = seq_ratings_data[col].apply(len)
    result[col + '_mean'] = seq_ratings_data[col].apply(len).mean()
    result[col + '_std'] = seq_ratings_len.max()
    result[col + '_min'] = seq_ratings_len.min()

In [ ]:
result

In [ ]:
def to_seq_data(
        ratings_data: pd.DataFrame,
    ) -> pd.DataFrame:

        ratings_data_transformed = ratings_data

        ratings_data_transformed.item_ids = ratings_data_transformed.item_ids.apply(
            lambda x: ",".join([str(i) for i in x])
        )
        ratings_data_transformed.ratings = ratings_data_transformed.ratings.apply(
            lambda x: ",".join([str(i) for i in x])
        )
        ratings_data_transformed.timestamps = ratings_data_transformed.timestamps.apply(
            lambda x: ",".join([str(i) for i in x])
        )
        ratings_data_transformed.rename(
            columns={
                "item_ids": "sequence_item_ids",
                "ratings": "sequence_ratings",
                "timestamps": "sequence_timestamps",
            },
            inplace=True,
        )
        return ratings_data_transformed

In [ ]:
seq_ratings_data = to_seq_data(seq_ratings_data)

In [ ]:
seq_ratings_data.head()

In [ ]:
len(set(ratings['item_id'].values))